In [90]:
import pymysql
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "",
    "db": "news",
    "charset": "utf8"
}

try:
    conn = pymysql.connect(**db_settings)
except Exception as ex:
    print(ex)

In [91]:
from selenium import webdriver
import requests
import time
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException

In [92]:
path = r'C:\\Users\\Administrator\\Downloads\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.maximize_window()

driver.get("https://www.facebook.com/") # 更改網址以前往不同網頁
print(driver.title)

# 登入
time.sleep(1)
username = driver.find_element_by_id('email')
username.send_keys('annie881112@yahoo.com.tw')

password = driver.find_element_by_id('pass')
password.send_keys('annie1329')

button = driver.find_element_by_name('login')
button.click()

Facebook - 登入或註冊


In [93]:
# 跳到東森新聞頁面
time.sleep(2)
driver.get("https://www.facebook.com/news.ebc")
print(driver.title)

(2) 東森新聞 | Facebook


In [94]:
count = 0
time.sleep(2)
i = 0
while(True and count < 50):
    try:
        content_more = driver.find_elements_by_class_name('abiwlrkh.gpro0wi8.m9osqain.buofh1pr')[i]
        
        if('隱藏' in content_more.text):
            i += 1
            continue

        content_more.click()
        count += 1
        print(str(count) + ' ' + content_more.text)
        time.sleep(0.5)
    except StaleElementReferenceException: #element is not attached to the page document
        #define the web element once again
        continue
    except ElementNotInteractableException:
        continue
    except ElementClickInterceptedException:
        i += 1
        continue
    except NoSuchElementException as e: #點完所有更多留言按鈕了
        
        # 滾動網頁/下拉視窗(scroll down) 
        #function4:scroll down window #function4:向下捲動視窗
        print('function4:scroll down window')
        driver.execute_script('')
    
i = 0
while(i < 5):
    driver.execute_script('')
    i+=1

IndexError: list index out of range

In [89]:
all = [] #存放所有留言
url = ""
soup = bs(driver.page_source, 'html.parser')
newsblock = soup.find_all("div", {"class": "du4w35lb k4urcfbm l9j0dhe7 sjgh65i0"})   # 每個新聞的block
for detial in newsblock:
    print('\n\n=================================================')
    url = detial.find('div', {'class': 'j83agx80 soycq5t1 ni8dbmo4 stjgntxs l9j0dhe7'})
    if(url != None):
        url = url.find('a')
    news_id = ''
    if(url != None):
        url = url['href']
        print(url)
        if('NewsID%3D' in url):
            news_id = url.split('NewsID%3D')[1]
            news_id = news_id.split('%')[0]
            
            print('newID: ' + news_id)
    
    # 確認該新聞是否在DB中
    with conn.cursor() as cursor:
      sql = 'SELECT News_id FROM original WHERE News_id = %s AND Provenance = \'東森\''
      cursor.execute(sql, (news_id))
      result = cursor.fetchall()
      check = None
      for row in result:
        check = row[0]
    
    if(check == None and url != None):     #該新聞有連結且不在DB中
        # # 抓取內容
        # r2 = requests.get(url)
        # soup2 = bs(r2.text, "lxml")
        # a2 = soup2.find("div", {"class"": "fncnews-content"})
        # #標題
        # title = ''
        # title = soup2.find("h1").text
        # #url
        # url = ''
        # url = soup2.find("link", {'rel': 'canonical'})['href']
        # print(url)

        
        # # 日期
        # date = soup2.find("div", {"class": "page-title-text"})
        # date = date.find('time', {'class': 'page-date'})
        # date = date.text
        # print(date)
        # print(url)
        # for content_temp in (a2.findAll("p")):
        #     if(('▲' in content_temp.text)):     #跳過圖片註解
        #         continue
        #     if((('/' in content_temp.text) or ('／' in content_temp.text)) and ('報導' in content_temp.text)):
        #         continue
        #     content += content_temp.text
        # #print(content)

        # with conn.cursor() as cursor:
        #     # 新增資料SQL語法
        #     command = 'INSERT INTO original VALUES(%s, %s, %s, %s, %s, %s, %s)'
        #     cursor.execute(command, (news_id, date, title, url, number[i], '三立', content))
        #     # 儲存變更
        # conn.commit()
        continue
    
    elif(check == None):
        continue

    #------------------------------------------------------------------------------------------
    print('\n留言：')
    comments = detial.find_all('div', {'class': 'ecm0bbzt e5nlhep0 a8c37x1j'})
    for i in comments:
        comment = ""
        for j in i.find_all('div', {'class': 'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql'}):         #每則留言
            for k in j.find_all('div'):         #每一行，可能包含標註人名
                temp = k.find('a')
                if(temp != None):       #有人名
                    temp = temp.text
                    comment += (k.text.split(temp))[1]
                else:           #沒人名
                    comment += (k.text + '。')
            print(comment)
            # with conn.cursor() as cursor:
            #     # 新增資料SQL語法
            #     command = 'INSERT INTO comments(News_id, Provenance, Comment) VALUES(%s, %s, %s)'
            #     cursor.execute(command, (news_id, '東森', comment))
            #     # 儲存變更
            #     conn.commit()
        print('----------------------------------------------\n')





https://l.facebook.com/l.php?u=https%3A%2F%2Fnews.ebc.net.tw%2Fnews%2Fentertainment%2F269494%3Ffbclid%3DIwAR3rX4wM-6jLhkjXQYnRVCHLbkgPn1cdS1dhwIY0uqWbp_LAYmQChlyrKvc&h=AT0FsQEq4AHWx59KQzjCBBkkSUJmvOdJjpUce1NSFjJROSqfszwgmyyGkXClpIFEqa6v36aZSZJ5l_wSA2TaSzu91yMrZ0wFk6J3bQfunwj11Xb4vquEjkQICPo9Wbo5xRj5&__tn__=H-R&c[0]=AT2_MZbQL5yTk_K2bLuvPyEwtRtODF1IVj3ZKbtKp09RlLzz1E49z9YU_KZnJcLTpD5B_3E6DF6bHxMtCbxQINirpHKuEQ0qHxyUkqFcCPHINCkztJPzviNJPeeVLNCY_jN40iGf7LdAjxLLYoLsT2X_TnFW5ULovgxPUgVYxl_hu_o






https://l.facebook.com/l.php?u=https%3A%2F%2Fnews.ebc.net.tw%2Fnews%2Fliving%2F269497%3Ffbclid%3DIwAR1GrRZeWKv-7tgkEVbS-lN3CgZVAv6d3dUdH6KdWgKCOoUJWvAogZlVqWQ&h=AT0oMhnp9_bK1XCvhF0IrGe1TGPw4QvD8hcfsQlrJbHugZeuSeipYhZip1ZVvyYLaiuSUqFDthddQAL9tXp7MFKULlYEkHZOeX92t5kHEScSFWyWxTdKg3rPO8En0zmPugTb&__tn__=H-R&c[0]=AT19FoEPVxS20IoTb8i-rkercka2B-8WsBRQFLAso_DjQIOeiLsCTCTudHzg36qYU_v0jZS5vNomDLzauklAkhnUU2406J-LZyHaH34O-yEPHcKMGO0KAyMr8-vyyrES92otMT1jFFH-7b_ey_F8jNuUKlNDEZEYV1V6eVLkhDV6u8A


https://l